# **Pose estimation and counter**

In [1]:
!pip install mediapipe opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

Video feed

In [3]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 1. Make Decisions

In [4]:
cap = cv2.VideoCapture(0)

#Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks (image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                               mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                               mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )



        cv2.imshow('Mediapipe Feed', image)
    
        if cv2.waitKey(10) & 0xFF ==ord('q'):
             break

cap.release()
cv2.destroyAllWindows()

C:\Users\Arnab\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


# 2. Determining joints

In [5]:
cap = cv2.VideoCapture(0)

#Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except: 
            pass
            

        # Render detections
        mp_drawing.draw_landmarks (image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                               mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), #for dots
                               mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) #for lines
                                  )



        cv2.imshow('Mediapipe Feed', image)
    
        if cv2.waitKey(10) & 0xFF ==ord('q'):
             break

cap.release()
cv2.destroyAllWindows()

[x: 0.51592046
y: 0.704513848
z: -0.862042308
visibility: 0.999413729
, x: 0.513487518
y: 0.65539664
z: -0.84638232
visibility: 0.999096513
, x: 0.522156954
y: 0.653472662
z: -0.846343875
visibility: 0.998870075
, x: 0.530983508
y: 0.651716888
z: -0.846533954
visibility: 0.998825371
, x: 0.485719204
y: 0.661033
z: -0.872773528
visibility: 0.999336183
, x: 0.473408163
y: 0.663093567
z: -0.872996151
visibility: 0.999224544
, x: 0.461060107
y: 0.665172696
z: -0.873263478
visibility: 0.999182403
, x: 0.521157861
y: 0.675587535
z: -0.659268498
visibility: 0.998988926
, x: 0.433822334
y: 0.696543276
z: -0.770888
visibility: 0.999383926
, x: 0.508241475
y: 0.754155457
z: -0.780657172
visibility: 0.999175847
, x: 0.464232028
y: 0.763133347
z: -0.81411171
visibility: 0.999486327
, x: 0.606272221
y: 0.887242436
z: -0.472693056
visibility: 0.988649607
, x: 0.355558336
y: 0.956416965
z: -0.584883511
visibility: 0.985312879
, x: 0.723423779
y: 1.09213817
z: -0.276564479
visibility: 0.345293522
, x:

In [6]:
len(landmarks)

33

In [7]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [8]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.611578584
y: 0.932002425
z: -0.572983742
visibility: 0.989971519

In [9]:
mp_pose.PoseLandmark.LEFT_SHOULDER

<PoseLandmark.LEFT_SHOULDER: 11>

In [10]:
mp_pose.PoseLandmark.LEFT_SHOULDER.value #giving the specific landmarks

11

In [11]:
mp_pose.PoseLandmark.RIGHT_WRIST.value # Searching for index of any landmark in the body

16

In [12]:
mp_pose.PoseLandmark.LEFT_ELBOW.value

13

# 3. Calculating Angles

In [13]:
def calculate_angle(a,b,c):
    a= np.array(a) #First
    b= np.array(b) #mid
    c= np.array(c) #End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle

    return angle

In [14]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [15]:
shoulder, elbow, wrist

([0.6115785837173462, 0.9320024251937866],
 [0.7614896297454834, 1.1660879850387573],
 [0.799742579460144, 1.3479408025741577])

In [16]:
calculate_angle(shoulder, elbow, wrist)

159.24308806743588

In [17]:
tuple(np.multiply(elbow,[640, 480]).astype(int))

(487, 559)

In [18]:
cap = cv2.VideoCapture(0)

#Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        #Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            #Get Coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            #Get angles
            angle = calculate_angle(shoulder, elbow, wrist)
            
            #Visualize
            cv2.putText(image, str(angle), (int(elbow[0] * 640), int(elbow[1] * 480)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            print(landmarks)
        except: 
            pass
            

        # Render detections
        mp_drawing.draw_landmarks (image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                               mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                               mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )



        cv2.imshow('Mediapipe Feed', image)
    
        if cv2.waitKey(10) & 0xFF ==ord('q'):
             break

cap.release()
cv2.destroyAllWindows()

# Curl counter

In [20]:
cap = cv2.VideoCapture(0)

#Curl counter variables
counter = 0
stage = None

#Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        #Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            #Get Coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            #Get angles
            angle = calculate_angle(shoulder, elbow, wrist)
            
            #Visualize
            cv2.putText(image, str(angle), (int(elbow[0] * 640), int(elbow[1] * 480)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            #Curl counter logic
            if angle >160:
                stage = "    Down"
            elif angle<30 and stage == 'Down':
                stage="    Up"
                counter +=1
                print(counter)
        
        except: 
            pass

        #Render Curl counter
        #Setup status box
        cv2.rectangle(image,(0,0), (225,73), (245,117,16), -1)


        #Creating Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)   

        #Creating Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)   

        # Render detections
        mp_drawing.draw_landmarks (image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                               mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                               mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )



        cv2.imshow('Mediapipe Feed', image)
    
        if cv2.waitKey(10) & 0xFF ==ord('q'):
             break

cap.release()
cv2.destroyAllWindows()